### Import libraries 

In [1]:
import pandas as pd
import numpy as np

import statsmodels.api as sm

from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import StandardScaler

import joblib

%matplotlib inline


### EDA

In [2]:
train = pd.read_csv('data/test.csv')

/var/folders/fk/tl38z_650pnb8d8bzsdfs50h0000gn/T/ipykernel_32095/1830354490.py:1: DtypeWarning: Columns (40) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('data/test.csv')


In [3]:
# to create a record of the original list of ID before we do data processing
# this is to ensure we do not loose the original set of IDs that we will require for submission to kaggle
test_original_id = train[['id']].copy()
test_original_id.head()
len(test_original_id)

16737

In [4]:
#changing all heading to lowercase
train = train.rename(columns=str.lower)

In [5]:
# filling in missing postal codes for two addresses in the data set
train.loc[train['address'] == '215, CHOA CHU KANG CTRL', 'postal'] = '680215'
train.loc[train['address'] == '238, COMPASSVALE WALK', 'postal'] = '540238'

In [6]:
# create new column for hdb_age_when_sold
# this is to show the remaining years left on the lease as of the year transacted
hdb_age_when_sold = train["tranc_year"] - train["lease_commence_date"]
train.insert(loc = 9, column = "hdb_age_when_sold", value = hdb_age_when_sold)

In [7]:
# drop all other columns for age of the flat that will no longer be used
train.drop(columns=['tranc_yearmonth', 'lease_commence_date', 'year_completed', 'hdb_age'],
       inplace=True)

In [8]:
#dropping the all except town for location pinpointing
train.drop(columns=['block', 'street_name', 'address', 'planning_area','postal'], 
       inplace=True)

In [9]:
# drop the columns with latitudes and longitudes
train.drop(columns=['latitude', 'longitude', 'mrt_latitude', 'mrt_longitude', 'bus_stop_latitude',
       'bus_stop_longitude', 'pri_sch_latitude', 'pri_sch_longitude', 'sec_sch_latitude', 'sec_sch_longitude'], 
       inplace=True)

In [10]:
# keep mid_storey to represent the location of the flat in the block of flats
train.drop(columns=['storey_range','lower','upper','mid'], 
       inplace=True)

In [11]:
# drop full flat type and sqm
train.drop(columns=['floor_area_sqm', 'full_flat_type'], 
       inplace=True)

In [12]:
# compute new boolean column for whether flat has rental units
train['has_rental'] = (train[['1room_rental', '2room_rental', '3room_rental', 'other_room_rental']].sum(axis=1) > 0).astype(int) 

In [13]:
# drop all sold and all rental columns (except the new has_rental column)
train.drop(columns=['1room_sold','2room_sold','3room_sold','4room_sold','5room_sold','exec_sold','multigen_sold','studio_apartment_sold', '1room_rental', '2room_rental', '3room_rental', 'other_room_rental'], 
       inplace=True)
train.insert(16, 'has_rental', train.pop('has_rental'))

In [14]:
# drop residential
train.drop(columns=['residential'], 
       inplace=True)

In [15]:
# dropping columns for hawker and malls
train.drop(columns=['mall_within_500m', 'mall_within_1km', 'hawker_within_500m', 'hawker_within_1km', 'market_hawker'],
       inplace=True)

In [16]:
# dropping all other irrelevant columns
train.drop(columns=['mrt_name', 'bus_stop_name', 'pri_sch_name', 'sec_sch_name'],
       inplace=True)

In [17]:
#filling in missing values 
train['mall_within_2km'] = train['mall_within_2km'].fillna(value=0)
train['hawker_within_2km'] = train['hawker_within_2km'].fillna(value=0)
train.dropna(subset = ['mall_nearest_distance'], inplace=True)

In [18]:
#changing type from int to object
train['tranc_month'] = train['tranc_month'].astype(object)

#changing type from object to bool
train['commercial'] = train['commercial'].map({'Y': 1, 'N': 0})
train['multistorey_carpark'] = train['multistorey_carpark'].map({'Y': 1, 'N': 0})
train['precinct_pavilion'] = train['precinct_pavilion'].map({'Y': 1, 'N': 0})


In [19]:
# filtering all terrace houses from the data set
filtered_all_terrace = train[train['flat_model'] == 'Terrace']
filtered_all_terrace.shape


(10, 31)

In [20]:
# dropping all terrace houses from the data set
train = train.drop(filtered_all_terrace.index)
train.shape

(16643, 31)

In [21]:
# mapping the town to the region and tagging whether it is mature or non-mature estate
town_region_mapping = {'SEMBAWANG': 'NORTH',
                  'WOODLANDS': 'NORTH',
                  'YISHUN': 'NORTH',
                  'ANG MO KIO': 'NORTH-EAST MATURE',
                  'HOUGANG': 'NORTH-EAST',
                  'PUNGGOL': 'NORTH-EAST',
                  'SENGKANG': 'NORTH-EAST',
                  'SERANGOON': 'NORTH-EAST MATURE',
                  'BEDOK': 'EAST MATURE',
                  'PASIR RIS': 'EAST MATURE',
                  'TAMPINES': 'EAST MATURE',
                  'BUKIT BATOK': 'WEST',
                  'BUKIT PANJANG': 'WEST',
                  'CHOA CHU KANG': 'WEST',
                  'CLEMENTI': 'WEST MATURE',
                  'JURONG EAST': 'WEST',
                  'JURONG WEST': 'WEST',
                  'BISHAN': 'CENTRAL MATURE',
                  'BUKIT MERAH': 'CENTRAL MATURE',
                  'BUKIT TIMAH': 'CENTRAL MATURE',
                  'CENTRAL AREA': 'CENTRAL MATURE',
                  'GEYLANG': 'CENTRAL MATURE',
                  'KALLANG/WHAMPOA': 'CENTRAL MATURE',
                  'MARINE PARADE': 'CENTRAL MATURE',
                  'QUEENSTOWN': 'CENTRAL MATURE',
                  'TOA PAYOH': 'CENTRAL MATURE'
                }
train['region_maturity'] = np.vectorize(town_region_mapping.get)(train['town'])

#reorganising the new column in sequence for clarity
train.insert(0, 'region_maturity', train.pop('region_maturity'))

In [22]:
# defining the different categories 
standard_models = ['Standard', 'New Generation', 'Simplified', 'Model A', 'Model A2', 'Improved', 'Apartment', 'Premium Apartment']
maisonette_models = ['Maisonette', 'Improved-Maisonette', 'Model A-Maisonette', 'Premium Maisonette', 'Premium Apartment Loft']
private_design_models = ['DBSS', 'Type S1', 'Type S2']

#assigning the different rows in the df to their respective categories
conditions = [
    train['flat_model'].isin(standard_models),
    train['flat_model'].isin(maisonette_models),
    train['flat_model'].isin(private_design_models)
]

#creating a new column called model_category containing the newly defined categories
values = ['Standard', 'Maisonette', 'Private Design']
train['model_category'] = np.select(conditions, values, default='Unknown')

#reorganising the new column in sequence for clarity
train.insert(4, 'model_category', train.pop('model_category'))


In [23]:
#dropping rows with the unused flat model (adjoined and 2-room)
filtered_adj_2r = train[(train['flat_model'] == 'Adjoined flat') | (train['flat_model'] == '2-room')]
train = train.drop(filtered_adj_2r.index)

#dropping rows with the unused flat types (1 room and multi-gen)
filtered_multigen_1r = train[(train['flat_type'] == '1 ROOM') | (train['flat_type'] == 'MULTI-GENERATION')]
train = train.drop(filtered_multigen_1r.index)

#dropping the unused columns flat_model, town 
train.drop(columns = ['flat_model','town'], inplace=True)


In [24]:
#creating column units_per_floor
train['units_per_floor'] = train['total_dwelling_units'] / train['max_floor_lvl']

#insert column units_per_floor at index 3
train.insert(3, 'units_per_floor', train.pop('units_per_floor'))

#dropping the previously engineered columns
train.drop(columns = ['total_dwelling_units','max_floor_lvl'], inplace=True)


In [25]:
#dummifying categorical variables and one-hot encoding
dummies_cols = ['region_maturity', 'flat_type', 'model_category','tranc_month']
train_dummies = train.copy()
for col in dummies_cols:
    train_dummies = pd.get_dummies(train_dummies, columns=[col], prefix=col, drop_first=True)
train_dummies.columns

Index(['id', 'units_per_floor', 'hdb_age_when_sold', 'tranc_year',
       'mid_storey', 'floor_area_sqft', 'commercial', 'multistorey_carpark',
       'precinct_pavilion', 'has_rental', 'mall_nearest_distance',
       'mall_within_2km', 'hawker_nearest_distance', 'hawker_within_2km',
       'hawker_food_stalls', 'hawker_market_stalls', 'mrt_nearest_distance',
       'bus_interchange', 'mrt_interchange', 'bus_stop_nearest_distance',
       'pri_sch_nearest_distance', 'vacancy', 'pri_sch_affiliation',
       'sec_sch_nearest_dist', 'cutoff_point', 'affiliation',
       'region_maturity_EAST MATURE', 'region_maturity_NORTH',
       'region_maturity_NORTH-EAST', 'region_maturity_NORTH-EAST MATURE',
       'region_maturity_WEST', 'region_maturity_WEST MATURE',
       'flat_type_3 ROOM', 'flat_type_4 ROOM', 'flat_type_5 ROOM',
       'flat_type_EXECUTIVE', 'model_category_Private Design',
       'model_category_Standard', 'tranc_month_2', 'tranc_month_3',
       'tranc_month_4', 'tranc_month

### Applying the cleaned data test set to the saved model

In [26]:
# create a record of the post-processed list of ID before we do modelling
# for us to merge the predicted resale prices to the respective IDs 
test_final_id = train_dummies[['id']].copy()

In [27]:
#define X matrix
X = train_dummies.drop(columns='id') #STOP HERE, include scale on test data, after scaling, this is X, then predict for y

#scale the values using StandardScaler()
#this should only be done on continuous numerical columns as defined in the above list

# Initialize the StandardScaler
ss = StandardScaler()

# Transform the scaler on the training data for numerical columns
numerical_cols = ['units_per_floor','hdb_age_when_sold','tranc_year','mid_storey','floor_area_sqft','mall_nearest_distance','mall_within_2km','hawker_nearest_distance','hawker_within_2km',
                  'hawker_food_stalls','hawker_market_stalls','mrt_nearest_distance','bus_stop_nearest_distance','pri_sch_nearest_distance','vacancy','sec_sch_nearest_dist','cutoff_point']

X[numerical_cols] = ss.fit_transform(X[numerical_cols])

In [28]:
# Load the trained model
ridge_cv = joblib.load("./trained.pkl") 

# create y_preds
# use the same variable name of the model saved in .pkl
y_preds_ridge = ridge_cv.predict(X)
print(y_preds_ridge)

# round y_preds_ridge to the nearest thousand
y_preds_ridge = np.round(y_preds_ridge,-3)

# convert it to a pd series
rounded_predicted_prices = pd.Series(y_preds_ridge, name='predicted')

[339791.02082323 505581.93271814 363370.74330872 ... 411946.59958448
 467060.41805439 377709.9161734 ]


In [29]:
# # concat the predicted prices to the test_final_id
# df_predicted = pd.concat([test_final_id, rounded_predicted_prices], axis=1)

# # merge final result back to the original list of IDs
# final_result = pd.merge(test_original_id, df_predicted, on='id', how='left')

# # check for nan values and fill them with 0
# final_result.isnull().sum()
# final_result.fillna(0, inplace=True)

# add the predicted values as a new column to the test_final_id
test_final_id["Predicted"] = y_preds_ridge
# merge final result back to the original list of IDs
final_result = pd.merge(test_original_id, test_final_id, on='id', how='left')
# check for nan values and fill them with 0
final_result.isnull().sum()
final_result.fillna(0, inplace=True)

In [30]:
final_result.head()

,id,Predicted
0,114982,340000.0
1,95653,506000.0
2,40303,363000.0
3,109506,291000.0
4,100149,421000.0


### Creating a CSV file from a newly created dataframe containing predicted resale prices and IDs

In [31]:
# save them to CSV file in the desired location
final_result.to_csv('./data/prediction_results.csv', index=False)